<a href="https://colab.research.google.com/github/Mizrahi845/My-first-repository-on-GitHub/blob/main/mor_mizrahi_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [127]:
(ds_train, ds_test), ds_info = tfds.load('mnist',
          split=['train', 'test'],
          shuffle_files=True,
          as_supervised=True,
          with_info=True,
)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples


print(num_of_classes)
print(image_shape)
print(train_size)
print(test_size)






10
(28, 28, 1)
60000
10000


In [128]:
def normalize_img(image, label):
    "Normalizes images: uint8 -> float32"
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
     normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Simple network sigmo

In [129]:
layers = [
        tf.keras.layers.Flatten(input_shape=image_shape),

        tf.keras.layers.Dense(32),
        tf.keras.layers.Activation('relu'),

        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
         ]

In [135]:
layers = [
        tf.keras.layers.Flatten(input_shape=image_shape),


      tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.002)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.005)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
]

In [136]:
layers = [
        tf.keras.layers.Flatten(input_shape=image_shape),

        tf.keras.layers.Dense(32),
        tf.keras.layers.Activation('sigmoid'),

        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
         ]

In [137]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_46 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_168 (Dense)                    │ (None, 32)                  │          25,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_121 (Activation)          │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_169 (Dense)                    │ (None, 10)                  │             330 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_42 (Softmax)                 │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,450 (99.41 KB)

 Trainable params: 25,450 (99.41 KB)

 Non-trainable params: 0 (0.00 B)

In [138]:
model.fit(ds_train,
          epochs=35,
          validation_data=ds_test)

Epoch 1/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8730 - sparse_categorical_accuracy: 0.8099 - val_loss: 0.2823 - val_sparse_categorical_accuracy: 0.9191
Epoch 2/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.2701 - sparse_categorical_accuracy: 0.9236 - val_loss: 0.2216 - val_sparse_categorical_accuracy: 0.9354
Epoch 3/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.2161 - sparse_categorical_accuracy: 0.9361 - val_loss: 0.1946 - val_sparse_categorical_accuracy: 0.9421
Epoch 4/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.1910 - sparse_categorical_accuracy: 0.9437 - val_loss: 0.1799 - val_sparse_categorical_accuracy: 0.9473
Epoch 5/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.1735 - sparse_categorical_accuracy: 0.9497 - val_loss: 0.1697 - val_sparse_categorical_accuracy: 0.9484
Epoch 6/35
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1565 - sparse_categorical_accuracy: 0.9550 - val_loss: 0.1616 - val_sparse_categorical_accuracy: